In [ ]:
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow as pa
import xgboost as xgb
import torch
import gc
import numpy as np

## Giải Phóng Bộ Nhớ GPU (Nếu Có)

Đoạn mã này định nghĩa một hàm `clear_gpu_memory()` và sau đó gọi hàm đó để giải phóng bộ nhớ GPU (Graphics Processing Unit) nếu GPU có sẵn. Điều này rất quan trọng trong các ứng dụng sử dụng GPU, đặc biệt là các ứng dụng liên quan đến deep learning, để tránh tình trạng hết bộ nhớ (out-of-memory error).

In [ ]:
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

clear_gpu_memory()

## Thiết Lập Đường Dẫn Tệp Parquet, Danh Sách Sản Phẩm và Các Lagged Feature

Đoạn mã này thực hiện các bước sau:

1.  **Định nghĩa đường dẫn đến tệp Parquet:** Xác định vị trí của tệp Parquet chứa dữ liệu huấn luyện và kiểm tra.
2.  **Đọc tệp Parquet:** Sử dụng thư viện `pyarrow.parquet` để đọc tệp Parquet vào một biến có tên `table`. Tệp Parquet là một định dạng lưu trữ cột hiệu quả, thường được sử dụng cho các tập dữ liệu lớn.
3.  **Định nghĩa danh sách sản phẩm:** Tạo một danh sách các tên cột, mỗi cột đại diện cho một sản phẩm khác nhau. Các tên cột này có vẻ như là các biến chỉ báo cho sự hiện diện của một sản phẩm cụ thể cho một khách hàng.
4.  **Tạo danh sách các Lagged Feature:** Tạo một danh sách các tên Lagged Feature. Lagged Feature là giá trị của một biến tại một thời điểm trước đó. Trong trường hợp này, nó tạo tên cho các Lagged Feature cho mỗi sản phẩm với độ trễ từ 1 đến 6 tháng.

**Các Bước Chi Tiết:**

*   **`parquet_file = "/kaggle/input/traintestparquet/train_test.parquet"`:** Gán đường dẫn đến tệp Parquet cho biến `parquet_file`.
*   **`table = pq.read_table(parquet_file)`:** Sử dụng hàm `pq.read_table()` từ thư viện `pyarrow.parquet` để đọc tệp Parquet vào một biến có tên `table`.
*   **`products = [...]`:** Tạo một danh sách các chuỗi, mỗi chuỗi đại diện cho tên của một sản phẩm. Các sản phẩm này dường như là các sản phẩm tài chính.
*   **`lag_months = [1, 2, 3, 4, 5, 6]`:** Tạo một danh sách các số nguyên, mỗi số nguyên đại diện cho một độ trễ (lag) tính bằng tháng.
*   **`lagged_features = [f"lag_{lag}_{prod}" for lag in lag_months for prod in products]`:** Sử dụng list comprehension để tạo một danh sách các tên Lagged Feature.  Đầu ra của đoạn code này là một danh sách các chuỗi, mỗi chuỗi có định dạng "lag\_\[lag]\_\[prod]", trong đó \[lag] là một giá trị từ `lag_months` và \[prod] là một giá trị từ `products`.  Ví dụ: "lag\_1\_ind\_ahor\_fin\_ult1", "lag\_2\_ind\_ahor\_fin\_ult1", ..., "lag\_6\_ind\_recibo\_ult1".

**Chú Thích Thuật Ngữ:**

*   **Parquet:** Một định dạng lưu trữ cột hiệu quả, thường được sử dụng cho các tập dữ liệu lớn.
*   **Lagged Feature:** Một đặc trưng được tạo bằng cách lấy giá trị của một biến tại một thời điểm trước đó.  Chúng thường được sử dụng trong các mô hình dự đoán chuỗi thời gian.
*   **List Comprehension:** Một cách ngắn gọn để tạo một danh sách mới từ một hoặc nhiều danh sách khác.

In [ ]:
parquet_file = "/kaggle/input/traintestparquet/train_test.parquet"
table = pq.read_table(parquet_file)


products = [
    "ind_ahor_fin_ult1", "ind_aval_fin_ult1", "ind_cco_fin_ult1",
    "ind_cder_fin_ult1", "ind_cno_fin_ult1", "ind_ctju_fin_ult1",
    "ind_ctma_fin_ult1", "ind_ctop_fin_ult1", "ind_ctpp_fin_ult1",
    "ind_deco_fin_ult1", "ind_deme_fin_ult1", "ind_dela_fin_ult1",
    "ind_ecue_fin_ult1", "ind_fond_fin_ult1", "ind_hip_fin_ult1",
    "ind_plan_fin_ult1", "ind_pres_fin_ult1", "ind_reca_fin_ult1",
    "ind_tjcr_fin_ult1", "ind_valo_fin_ult1", "ind_viv_fin_ult1",
    "ind_nomina_ult1", "ind_nom_pens_ult1", "ind_recibo_ult1"
]

lag_months = [1, 2, 3, 4, 5, 6]
lagged_features = [f"lag_{lag}_{prod}" for lag in lag_months for prod in products]

## Chia Tập Dữ Liệu Thành Train, Validation và Test

Đoạn mã này sử dụng thư viện `pyarrow` để chia tập dữ liệu `table` thành ba tập con: tập huấn luyện (train), tập kiểm định (validation), và tập kiểm tra (test) dựa trên giá trị của cột "month_int".

**Các Bước:**

1.  **Tạo Bộ Lọc (Filter):**
    *   `train_filter = pc.less(table["month_int"], 15)`: Tạo bộ lọc cho tập huấn luyện. Bản ghi được chọn vào tập huấn luyện nếu giá trị trong cột "month_int" nhỏ hơn 15.
    *   `val_filter = pc.equal(table["month_int"], 15)`: Tạo bộ lọc cho tập kiểm định. Bản ghi được chọn vào tập kiểm định nếu giá trị trong cột "month_int" bằng 15.
    *   `test_filter = pc.equal(table["month_int"], 16)`: Tạo bộ lọc cho tập kiểm tra. Bản ghi được chọn vào tập kiểm tra nếu giá trị trong cột "month_int" bằng 16.

2.  **Áp Dụng Bộ Lọc:**
    *   `train_table = table.filter(train_filter)`: Áp dụng bộ lọc `train_filter` để tạo tập huấn luyện `train_table`.
    *   `val_table = table.filter(val_filter)`: Áp dụng bộ lọc `val_filter` để tạo tập kiểm định `val_table`.
    *   `test_table = table.filter(test_filter)`: Áp dụng bộ lọc `test_filter` để tạo tập kiểm tra `test_table`.

**Ý Nghĩa:**

Đoạn mã chia dữ liệu theo thời gian, giả sử "month_int" đại diện cho tháng dưới dạng số nguyên. Dữ liệu từ các tháng trước tháng 15 được sử dụng để huấn luyện mô hình, dữ liệu tháng 15 được sử dụng để kiểm định mô hình (điều chỉnh các siêu tham số), và dữ liệu tháng 16 được sử dụng để đánh giá hiệu năng cuối cùng của mô hình.

**Chú Thích Thuật Ngữ:**

*   **pc (pyarrow.compute):** Mô-đun tính toán của thư viện `pyarrow`.
*   **Filter:** Một điều kiện được sử dụng để chọn một tập hợp con của dữ liệu.
*   **Validation Set:** Tập dữ liệu được sử dụng để điều chỉnh các siêu tham số của mô hình.


In [ ]:
# ------------------------------------------------------------------------
# 3. Chia train, val, test

train_filter = pc.less(table["month_int"], 15)
val_filter = pc.equal(table["month_int"], 15)
test_filter  = pc.equal(table["month_int"], 16)

train_table = table.filter(train_filter)
val_table  = table.filter(val_filter)
test_table  = table.filter(test_filter)

## Chuẩn Bị Dữ Liệu Huấn Luyện

Đoạn mã này chuẩn bị dữ liệu huấn luyện cho mô hình dự đoán sản phẩm mới.

**Các Bước:**

1.  **Xác định khách hàng mới sử dụng sản phẩm:** Lọc ra những khách hàng chuyển từ không sử dụng sang sử dụng sản phẩm trong tháng hiện tại.
2.  **Tạo ma trận đặc trưng:** Sử dụng các lagged feature (giá trị sản phẩm ở các tháng trước) cho những khách hàng này.
3.  **Tạo label:** Gán label cho mỗi khách hàng, tương ứng với sản phẩm mới mà họ sử dụng.
4.  **Kết hợp dữ liệu:** Ghép nối dữ liệu từ các sản phẩm khác nhau để tạo thành tập huấn luyện cuối cùng.

**Chú Thích Thuật Ngữ:**

*   **Lagged Feature:** Giá trị của một biến tại một thời điểm trong quá khứ.
*   **Label:** Nhãn, giá trị mục tiêu mà mô hình cần dự đoán.


In [ ]:
# ------------------------------------------------------------------------
# 4. Prepare training data

training_features_list = []
training_labels_list = []

for product_index, product in enumerate(products):
    current_month_indicators = train_table.column(product).to_numpy()
    previous_month_indicators = train_table.column(f"lag_1_{product}").to_numpy()

    newly_added_product_mask = (current_month_indicators == 1) & (previous_month_indicators == 0)
    if np.sum(newly_added_product_mask) == 0:
        continue

    lagged_feature_arrays = []
    for feature in lagged_features:
        feature_column = train_table.column(feature).to_numpy()
        lagged_feature_arrays.append(feature_column[newly_added_product_mask].reshape(-1, 1))

    feature_matrix = np.hstack(lagged_feature_arrays)
    training_features_list.append(feature_matrix)

    training_labels_list.append(np.full(feature_matrix.shape[0], product_index, dtype=np.int8))

if len(training_features_list) == 0:
    raise ValueError("No training data found. Check the data and filters.")

training_features = np.vstack(training_features_list)
training_labels = np.concatenate(training_labels_list)

print("Training samples:", training_features.shape[0])
print("Number of features:", training_features.shape[1])
print("Training labels distribution:", {i: int(np.sum(training_labels==i)) for i in np.unique(training_labels)})

## Chuẩn Bị Dữ Liệu Kiểm Định

Đoạn mã này chuẩn bị dữ liệu kiểm định, tương tự như quá trình chuẩn bị dữ liệu huấn luyện.

**Các Bước:**

1.  **Xác định khách hàng mới sử dụng sản phẩm:** Lọc ra những khách hàng chuyển từ không sử dụng sang sử dụng sản phẩm trong tháng hiện tại, sử dụng dữ liệu từ `val_table`.
2.  **Tạo Feature Matrix:** Sử dụng các lagged feature cho những khách hàng này.
3.  **Tạo label:** Gán label cho mỗi khách hàng, tương ứng với sản phẩm mới mà họ sử dụng.
4.  **Kết hợp dữ liệu:** Ghép nối dữ liệu từ các sản phẩm khác nhau để tạo thành tập kiểm định cuối cùng.

**Chú Thích Thuật Ngữ:**

*   **Lagged Feature:** Giá trị của một biến tại một thời điểm trong quá khứ.
*   **Feature Matrix:** Ma trận chứa các đặc trưng cho mỗi mẫu dữ liệu.
*   **Label:** Nhãn, giá trị mục tiêu mà mô hình cần dự đoán.


In [ ]:
# ------------------------------------------------------------------------
# 4.5. Prepare validation data

validation_features_list = []
validation_labels_list = []

for product_index, product in enumerate(products):
    current_month_indicators = val_table.column(product).to_numpy()
    previous_month_indicators = val_table.column(f"lag_1_{product}").to_numpy()

    newly_added_product_mask = (current_month_indicators == 1) & (previous_month_indicators == 0)
    if np.sum(newly_added_product_mask) == 0:
        continue

    lagged_feature_arrays = []
    for feature in lagged_features:
        feature_column = val_table.column(feature).to_numpy()
        lagged_feature_arrays.append(feature_column[newly_added_product_mask].reshape(-1, 1))

    feature_matrix = np.hstack(lagged_feature_arrays)
    validation_features_list.append(feature_matrix)

    validation_labels_list.append(np.full(feature_matrix.shape[0], product_index, dtype=np.int8))

if len(validation_features_list) == 0:
    print("Warning: No validation data found for new product additions. Validation set might be empty.")
    validation_features = np.empty((0, len(lagged_features)))
    validation_labels = np.empty(0, dtype=np.int8)
else:
    validation_features = np.vstack(validation_features_list)
    validation_labels = np.concatenate(validation_labels_list)

print("Validation samples:", validation_features.shape[0])
print("Number of features:", validation_features.shape[1])
print("Validation labels distribution:", {i: int(np.sum(validation_labels==i)) for i in np.unique(validation_labels)})

## Chuẩn Bị Dữ Liệu Kiểm Tra

Đoạn mã này chuẩn bị dữ liệu kiểm tra để đưa vào mô hình.

**Các Bước:**

1.  **Tạo ma trận đặc trưng:** Trích xuất các lagged feature từ `test_table` và ghép thành ma trận `test_features`.
2.  **Lấy mã khách hàng:** Trích xuất mã khách hàng (`ncodpers`) từ `test_table` để sử dụng sau này (ví dụ: tạo tệp nộp bài).

**Chú Thích Thuật Ngữ:**

*   **Lagged Feature:** Giá trị của một biến tại một thời điểm trong quá khứ.

In [ ]:
# ------------------------------------------------------------------------
# 5. Prepare test data

test_feature_arrays = []
for feature in lagged_features:
    arr = test_table.column(feature).to_numpy()
    test_feature_arrays.append(arr.reshape(-1, 1))
test_features = np.hstack(test_feature_arrays)

ncodpers = test_table.column("ncodpers").to_pylist()

print("Test samples:", test_features.shape[0])
print("Test feature shape:", test_features.shape)

Tạo XGBoost DMatrix và Thiết Lập Tham Số

Đoạn mã này chuẩn bị dữ liệu và cấu hình mô hình XGBoost.

**Các Bước:**

1.  **Tạo DMatrix:** Chuyển đổi dữ liệu huấn luyện, kiểm định và kiểm tra thành định dạng `DMatrix` mà XGBoost sử dụng, bao gồm `feature` và `label` (nếu có).
2.  **Thiết lập tham số:** Xác định các tham số để cấu hình mô hình XGBoost, bao gồm mục tiêu, tốc độ học, độ sâu tối đa của cây, và các tham số liên quan đến việc lấy mẫu cột.

**Chú Thích Thuật Ngữ:**

*   **DMatrix:** Cấu trúc dữ liệu được tối ưu hóa cho XGBoost.
*   **XGBoost:** Thuật toán gradient boosting được sử dụng để xây dựng mô hình.
*   **Feature:** Đặc trưng.
*   **Label:** Nhãn.


In [ ]:
# %%
# ------------------------------------------------------------------------
# 6. Build XGBoost DMatrix objects

dtrain = xgb.DMatrix(training_features, label=training_labels, feature_names=lagged_features)
dval = xgb.DMatrix(validation_features, label=validation_labels, feature_names=lagged_features) # Created dval
dtest  = xgb.DMatrix(test_features, feature_names=lagged_features)

# ------------------------------------------------------------------------
# 7. Set XGBoost parameters (multi:softprob for multiclass)
param = {
    "objective": "multi:softprob",
    "eta": 0.1,
    "min_child_weight": 10,
    "max_depth": 8,
    "silent": 1,
    "eval_metric": "mlogloss",
    "colsample_bytree": 0.8,
    "colsample_bylevel": 0.9,
    "num_class": len(products),
    "device": "cuda"
}

## Huấn Luyện Mô Hình XGBoost

Đoạn mã này thực hiện quá trình huấn luyện mô hình XGBoost.

**Các Bước:**

1.  **Thiết lập các tham số huấn luyện:**
    *   `num_boost_round`: Số lượng vòng lặp boosting (số lượng cây được xây dựng).
    *   `watchlist`: Danh sách các tập dữ liệu được sử dụng để theo dõi hiệu năng trong quá trình huấn luyện (trong trường hợp này, tập huấn luyện và tập kiểm định).
2.  **Huấn luyện mô hình:** Gọi hàm `xgb.train` để huấn luyện mô hình XGBoost với các tham số đã thiết lập, dữ liệu huấn luyện, và watchlist. Sử dụng `early_stopping_rounds` để dừng huấn luyện sớm nếu hiệu năng trên tập kiểm định không cải thiện trong một số vòng lặp nhất định.
3.  **In thông tin về độ quan trọng của feature:** Lấy và in ra thông tin về độ quan trọng của các `feature` trong mô hình đã huấn luyện. Các `feature` quan trọng hơn sẽ có điểm số cao hơn.

**Chú Thích Thuật Ngữ:**

*   **Boosting:** Một kỹ thuật machine learning kết hợp nhiều mô hình yếu để tạo ra một mô hình mạnh hơn.
*   **Feature Importance:** Độ quan trọng của một `feature` trong việc dự đoán.
*   **Feature:** Đặc trưng.
*   **Early Stopping:** Kỹ thuật dừng huấn luyện sớm nếu hiệu năng không cải thiện.


In [ ]:
# ------------------------------------------------------------------------
# 8. Train the XGBoost model

num_boost_round = 1000
watchlist = [(dtrain, "train"), (dval, "eval")]

print("Training XGBoost model ...")
model = xgb.train(param, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=20)

print("\nFeature Importance:")
feature_importance = model.get_fscore()  
for feat, score in sorted(feature_importance.items(), key=lambda item: item[1], reverse=True):
    print(f"{feat}: {score}")

## Dự Đoán Trên Dữ Liệu Kiểm Tra

Đoạn mã này sử dụng mô hình XGBoost đã được huấn luyện để dự đoán trên dữ liệu kiểm tra.

**Các Bước:**

1.  **Dự đoán:** Gọi hàm `model.predict` để tạo ra các dự đoán trên `DMatrix` của dữ liệu kiểm tra (`dtest`).
2.  **In hình dạng của kết quả dự đoán:** In ra hình dạng của mảng chứa các dự đoán.

**Chú Thích Thuật Ngữ:**

*   **DMatrix:** Cấu trúc dữ liệu được tối ưu hóa cho XGBoost.
*   **Predict:** Dự đoán, tạo ra các giá trị đầu ra từ mô hình dựa trên dữ liệu đầu vào.


In [ ]:
# -------------------------------------------------------------
# 9. Predict on test data (fixing the AttributeError)

preds = model.predict(dtest)
print("\nPredictions shape:", preds.shape)

## Định Nghĩa Hàm Tạo Tệp Submission

Đoạn mã này định nghĩa một hàm có tên `make_submission_file` để tạo tệp submission ở định dạng cần thiết cho cuộc thi.

**Các Bước:**

1.  **Viết header:** Ghi dòng tiêu đề "ncodpers,added\_products" vào tệp submission.
2.  **Lặp qua các dự đoán và mã khách hàng:** Với mỗi khách hàng,
    *   **Chọn top 7 sản phẩm được dự đoán:** Sắp xếp các sản phẩm theo xác suất dự đoán giảm dần và chọn 7 sản phẩm hàng đầu.
    *   **Chuyển đổi chỉ số sản phẩm thành tên sản phẩm:** Tìm tên của từng sản phẩm được dự đoán từ danh sách `products`.
    *   **Ghi dòng vào tệp submission:** Ghi mã khách hàng và danh sách các sản phẩm được dự đoán vào tệp submission.

**Chú Thích Thuật Ngữ:**

*   **Submission:** Tệp chứa các dự đoán của mô hình ở định dạng yêu cầu của cuộc thi hoặc hệ thống đánh giá.


In [ ]:
# -------------------------------------------------------------
# 10. Define submission function

import io

def make_submission_file(submission_file, predictions, customer_ids, products):
    submission_file.write("ncodpers,added_products\n".encode('utf-8'))
    for customer_id, prediction in zip(customer_ids, predictions):
        predicted_products_indices = np.argsort(prediction)[::-1][:7]
        predicted_product_names = [products[i] for i in predicted_products_indices]
        submission_file.write(("%s,%s\n" % (int(customer_id), " ".join(predicted_product_names))).encode('utf-8'))

## Định Nghĩa Hàm MAP@7 và Tạo Submission

Đoạn mã này định nghĩa hàm tính MAP@7 (Mean Average Precision at 7) và tạo tệp submission.

**Các Bước:**

1.  **Định nghĩa hàm tính MAP@7:**
    *   Định nghĩa hàm `apk(actual, predicted, k=7, default=0.0)` để tính Average Precision at k (AP@k) cho một khách hàng.
    *   Định nghĩa hàm `mapk(actual, predicted, k=7, default=0.0)` để tính Mean Average Precision at k (MAP@k) trên toàn bộ tập dữ liệu. Trong đoạn code này, k được đặt là 7 cho MAP@7.
2.  **Tạo tệp submission:**
    *   Sử dụng hàm `make_submission_file` (đã định nghĩa trước đó) để tạo tệp submission trong bộ nhớ (sử dụng `io.BytesIO`).
3.  **Tính MAP@7 (chỉ để minh họa):**
    *   Đọc tệp submission từ bộ nhớ.
    *   Trích xuất danh sách các sản phẩm được dự đoán cho mỗi khách hàng từ tệp submission.
    *   **Lưu ý quan trọng:** Trong đoạn code này, giá trị "thực tế" (actual) được đặt bằng giá trị dự đoán (predicted) để minh họa cách tính MAP@7. **Trong thực tế, bạn cần thay thế giá trị này bằng giá trị thực tế từ tập kiểm định (validation set) để có kết quả đánh giá chính xác.**
    *   Tính MAP@7 bằng cách sử dụng hàm `mapk` với danh sách các sản phẩm được dự đoán và danh sách các sản phẩm thực tế (đã được thay thế bằng giá trị dự đoán trong ví dụ này).
    *   In ra giá trị MAP@7.

**Chú Thích Thuật Ngữ:**

*   **MAP@7 (Mean Average Precision at 7):** Một metric đánh giá hiệu năng được sử dụng trong các bài toán gợi ý (recommendation), đo lường độ chính xác của 7 sản phẩm hàng đầu được dự đoán.
*   **Submission:** Tệp chứa các dự đoán của mô hình ở định dạng yêu cầu của cuộc thi hoặc hệ thống đánh giá.


In [ ]:
# -------------------------------------------------------------
# 11. Define MAP@7 metric function

def apk(actual, predicted, k=7, default=0.0):
    """
    Calculate the average precision at k (AP@k) for a single instance.
    
    :param actual: List of actual products.
    :param predicted: List of predicted products.
    :param k: The number of predictions to consider.
    :param default: The default value to return if there are no actual products.
    :return: The average precision at k.
    """
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    if not actual:
        return default
    return score / min(len(actual), k)

def mapk(actual, predicted, k=7, default=0.0):
    """
    Calculate the mean average precision at k (MAP@k) for a set of instances.
    
    :param actual: List of lists of actual products.
    :param predicted: List of lists of predicted products.
    :param k: The number of predictions to consider.
    :param default: The default value to return if there are no actual products.
    :return: The mean average precision at k.
    """
    return np.mean([apk(a, p, k, default) for a, p in zip(actual, predicted)])

## Ghi Tệp Submission Ra Định Dạng TXT và CSV

Đoạn mã này ghi nội dung của tệp submission (đã được tạo trong bộ nhớ) ra hai tệp vật lý: một tệp văn bản (`submission.txt`) và một tệp CSV (`submission.csv`).

**Các Bước:**

1.  **Lấy nội dung submission:** Lấy nội dung của submission từ `submission_io_buffer` (là một đối tượng `io.BytesIO` chứa nội dung submission ở dạng bytes) và giải mã nó thành chuỗi UTF-8.
2.  **Ghi vào tệp TXT:** Mở một tệp mới có tên `submission.txt` ở chế độ ghi (`'w'`) và ghi nội dung submission vào tệp này.
3.  **Ghi vào tệp CSV:** Mở một tệp mới có tên `submission.csv` ở chế độ ghi (`'w'`) và ghi nội dung submission vào tệp này.
4.  **In thông báo:** In ra thông báo xác nhận rằng tệp submission đã được ghi thành công.

**Ý Nghĩa:**

Đoạn mã này tạo hai bản sao của tệp submission ở hai định dạng khác nhau. Định dạng CSV thường được yêu cầu bởi các cuộc thi, trong khi định dạng TXT có thể hữu ích để xem và gỡ lỗi nội dung submission.

**Chú Thích Thuật Ngữ:**

*   **Submission:** Tệp chứa các dự đoán của mô hình ở định dạng yêu cầu của cuộc thi hoặc hệ thống đánh giá.
*   **UTF-8:** Một bảng mã ký tự phổ biến được sử dụng để biểu diễn văn bản.
*   **CSV (Comma Separated Values):** Một định dạng tệp văn bản đơn giản sử dụng dấu phẩy để phân tách các giá trị.


In [ ]:
# -------------------------------------------------------------
# 13. Write submission to txt and csv files

submission_content = submission_io_buffer.getvalue().decode('utf-8')

txt_filename = "submission.txt"
with open(txt_filename, 'w') as txt_file:
    txt_file.write(submission_content)
print(f"\nSubmission written to: {txt_filename}")

csv_filename = "submission.csv"
with open(csv_filename, 'w') as csv_file:
    csv_file.write(submission_content)
print(f"Submission written to: {csv_filename}")